In [1]:
# Initial imports
import os
import requests
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import datetime 
import yfinance as yf
import backtrader as bt
%matplotlib inline

#download csv as original df
og_betas_df = pd.read_csv(Path('../../../Betas.csv'))  
#create new df to leave original df alone. Set index to Ticker 
betas_df = og_betas_df.set_index('Ticker')

#display df
betas_df

,Beta 09/2012,Beta 09/2022,Beta 12/2008,Beta 9/2008
Ticker,,,,
NXPI,3.6130,1.4149,NaN,NaN
LULU,2.3932,1.2971,2.1419,1.8343
AMD,2.2121,1.9441,2.4404,2.5507
SIRI,2.1154,0.9241,2.5329,3.0197
ADSK,2.1115,1.4623,2.3257,2.1548
...,...,...,...,...
VRTX,0.3325,0.4960,1.0370,0.8088
VRSK,0.1337,0.8391,NaN,NaN
PANW,0.0165,1.2415,NaN,NaN


In [2]:
#display corralation of 2012 and 2022 betas
betas_df[["Beta 09/2012","Beta 09/2022"]].corr(method='pearson')

,Beta 09/2012,Beta 09/2022
Beta 09/2012,1.000000,0.395354
Beta 09/2022,0.395354,1.000000


In [3]:
#Separate df into 3 dfs
betas_df_2012 = betas_df.sort_values(by=['Beta 09/2012'])
betas_df_2012 = betas_df_2012[['Beta 09/2012']].dropna()
betas_df_12_2008 = betas_df.sort_values(by=['Beta 12/2008'])
betas_df_12_2008 = betas_df_12_2008[['Beta 12/2008']].dropna()
betas_df_9_2008 = betas_df.sort_values(by=['Beta 9/2008'])
betas_df_9_2008 = betas_df_9_2008[['Beta 9/2008']].dropna()

In [4]:
#Separate 3 dfs into 6 dataframes

#number 1
low_betas_df_2012 = betas_df_2012.head(10)
low_betas_df_2012

,Beta 09/2012
Ticker,
META,-0.1001
SPLK,0.0020
PANW,0.0165
VRSK,0.1337
VRTX,0.3325
DLTR,0.3475
XEL,0.3523
TSLA,0.3849
ORLY,0.3875


In [5]:
#number 2
high_betas_df_2012 = betas_df_2012.tail(10)
high_betas_df_2012

,Beta 09/2012
Ticker,
FTNT,1.7109
KLAC,1.7515
BIDU,1.8036
MELI,1.9264
ALGN,2.0641
ADSK,2.1115
SIRI,2.1154
AMD,2.2121
LULU,2.3932


In [6]:
#number 3
low_betas_df_12_2008 = betas_df_12_2008.head(10)
low_betas_df_12_2008

,Beta 12/2008
Ticker,
TMUS,0.1323
ODFL,0.2799
AZN,0.4290
AMGN,0.4416
DLTR,0.4623
GILD,0.5433
ADP,0.5536
XEL,0.5581
PEP,0.5757


In [7]:
#number 4
high_betas_df_12_2008 = betas_df_12_2008.tail(10)
high_betas_df_12_2008

,Beta 12/2008
Ticker,
LULU,2.1419
AMZN,2.1560
ALGN,2.1885
BIDU,2.2487
ADSK,2.3257
NVDA,2.3955
AMD,2.4404
SIRI,2.5329
DXCM,2.5423


In [8]:
#number 5
low_betas_df_9_2008 = betas_df_9_2008.head(10)
low_betas_df_9_2008

,Beta 9/2008
Ticker,
PEP,0.2107
TMUS,0.3286
ODFL,0.3764
ILMN,0.4548
IDXX,0.4742
MDLZ,0.5031
AZN,0.5178
BIIB,0.5308
ADP,0.5733


In [9]:
#number 6
high_betas_df_9_2008 = betas_df_9_2008.tail(10)
high_betas_df_9_2008

,Beta 9/2008
Ticker,
AMZN,2.2728
ALGN,2.3201
ASML,2.3883
DXCM,2.4084
AMD,2.5507
BIDU,2.6021
AAPL,2.6179
NVDA,2.9969
SIRI,3.0197


In [10]:
#merge both 2008 low beta dfs with outer join
low_betas_df_2008 = pd.merge(low_betas_df_12_2008,low_betas_df_9_2008, on='Ticker', how='outer')
low_betas_df_2008

,Beta 12/2008,Beta 9/2008
Ticker,,
TMUS,0.1323,0.3286
ODFL,0.2799,0.3764
AZN,0.4290,0.5178
AMGN,0.4416,NaN
DLTR,0.4623,NaN
GILD,0.5433,NaN
ADP,0.5536,0.5733
XEL,0.5581,NaN
PEP,0.5757,0.2107


In [11]:
#merge both 2008 high beta dfs with outer join
high_betas_df_2008 = pd.merge(high_betas_df_12_2008, high_betas_df_9_2008, on='Ticker', how='outer')
high_betas_df_2008

,Beta 12/2008,Beta 9/2008
Ticker,,
LULU,2.1419,NaN
AMZN,2.1560,2.2728
ALGN,2.1885,2.3201
BIDU,2.2487,2.6021
ADSK,2.3257,NaN
NVDA,2.3955,2.9969
AMD,2.4404,2.5507
SIRI,2.5329,3.0197
DXCM,2.5423,2.4084


In [12]:
# create list of tickers for 2008 low beta portfolio

low_betas_df_2008_tickers = low_betas_df_2008.index.tolist()
low_betas_df_2008_tickers

['TMUS',
 'ODFL',
 'AZN',
 'AMGN',
 'DLTR',
 'GILD',
 'ADP',
 'XEL',
 'PEP',
 'MDLZ',
 'ILMN',
 'IDXX',
 'BIIB',
 'ROST']

In [13]:
# create list of tickers for 2008 high beta portfolio
high_betas_df_2008_tickers = high_betas_df_2008.index.tolist()
high_betas_df_2008_tickers

['LULU',
 'AMZN',
 'ALGN',
 'BIDU',
 'ADSK',
 'NVDA',
 'AMD',
 'SIRI',
 'DXCM',
 'MELI',
 'ASML',
 'AAPL']

In [14]:
# create list of tickers for 2012 low beta portfolio

low_betas_df_2012_tickers = low_betas_df_2012.index.tolist()
low_betas_df_2012_tickers

['META', 'SPLK', 'PANW', 'VRSK', 'VRTX', 'DLTR', 'XEL', 'TSLA', 'ORLY', 'AMGN']

In [15]:
# create list of tickers for 2012 high beta portfolio

high_betas_df_2012_tickers = high_betas_df_2012.index.tolist()
high_betas_df_2012_tickers

['FTNT', 'KLAC', 'BIDU', 'MELI', 'ALGN', 'ADSK', 'SIRI', 'AMD', 'LULU', 'NXPI']

In [16]:
#create list of all tickers
tickers = list(set(low_betas_df_2008_tickers + high_betas_df_2008_tickers + low_betas_df_2012_tickers + high_betas_df_2012_tickers))
tickers

['TMUS',
 'IDXX',
 'AMD',
 'ADP',
 'BIIB',
 'META',
 'PEP',
 'ALGN',
 'PANW',
 'AAPL',
 'ROST',
 'DXCM',
 'AMZN',
 'VRSK',
 'AMGN',
 'ORLY',
 'VRTX',
 'TSLA',
 'LULU',
 'BIDU',
 'XEL',
 'NXPI',
 'ASML',
 'ILMN',
 'SIRI',
 'ADSK',
 'FTNT',
 'KLAC',
 'GILD',
 'DLTR',
 'MDLZ',
 'ODFL',
 'SPLK',
 'NVDA',
 'AZN',
 'MELI']

In [17]:
# Load stock data from 2008 to now
df_portfolio = yf.download(tickers, start="2008-01-01")

[*********************100%***********************]  36 of 36 completed


In [18]:
df_portfolio_2008 = df_portfolio.loc["2008-12-30":"2008-12-30"]
df_portfolio_2012 = df_portfolio.loc["2012-09-28":"2012-09-28"]
df_portfolio_2022 = df_portfolio.loc["2022-09-28":"2022-09-28"]
#test - find AAPL apple price 
float(df_portfolio_2008["Adj Close"]["AAPL"])

2.6313276290893555

In [19]:
# create list of 2008 low beta stock closing price
def price2008(list):
    new_list = []
    for ticker in list:
        data = [ticker, float(df_portfolio_2008["Adj Close"][ticker])]  
        new_list.append(data)
    print(new_list)

low_betas_price_2008 = price2008(low_betas_df_2008_tickers)
low_betas_price_2008

[['TMUS', 23.597026824951172], ['ODFL', 7.837845325469971], ['AZN', 11.013473510742188], ['AMGN', 43.46708297729492], ['DLTR', 13.609999656677246], ['GILD', 19.97959327697754], ['ADP', 24.37674331665039], ['XEL', 11.1924467086792], ['PEP', 36.602134704589844], ['MDLZ', 12.614680290222168], ['ILMN', 25.0], ['IDXX', 17.844999313354492], ['BIIB', 47.68000030517578], ['ROST', 6.433244705200195]]


In [20]:
# create list of 2008 low beta stock closing price
high_betas_price_2008 = price2008(high_betas_df_2008_tickers)
high_betas_price_2008

[['LULU', 3.734999895095825], ['AMZN', 2.5380001068115234], ['ALGN', 8.270000457763672], ['BIDU', 12.692999839782715], ['ADSK', 18.969999313354492], ['NVDA', 1.8405638933181763], ['AMD', 2.2100000381469727], ['SIRI', 0.10744978487491608], ['DXCM', 0.6299999952316284], ['MELI', 15.203573226928711], ['ASML', 12.36668872833252], ['AAPL', 2.6313276290893555]]


In [21]:
# create list of 2012 low beta stock closing price
def price2012(list):
    new_list = []
    for ticker in list:
        data = [ticker, float(df_portfolio_2012["Adj Close"][ticker])]  
        new_list.append(data)
    print(new_list)

low_betas_price_2012 = price2012(low_betas_df_2012_tickers)
low_betas_price_2012

[['META', 21.65999984741211], ['SPLK', 36.720001220703125], ['PANW', 20.523332595825195], ['VRSK', 46.451602935791016], ['VRTX', 55.88999938964844], ['DLTR', 48.290000915527344], ['XEL', 20.058334350585938], ['TSLA', 1.9520000219345093], ['ORLY', 83.62000274658203], ['AMGN', 65.2418212890625]]


In [22]:
# create list of 2012 high beta stock closing price
def price2012(list):
    new_list = []
    for ticker in list:
        data = [ticker, float(df_portfolio_2012["Adj Close"][ticker])]  
        new_list.append(data)
    print(new_list)

high_betas_price_2012 = price2012(high_betas_df_2012_tickers)
high_betas_price_2012

[['FTNT', 4.823999881744385], ['KLAC', 30.047407150268555], ['BIDU', 116.88999938964844], ['MELI', 80.65840911865234], ['ALGN', 36.970001220703125], ['ADSK', 33.349998474121094], ['SIRI', 2.319124460220337], ['AMD', 3.369999885559082], ['LULU', 73.94000244140625], ['NXPI', 23.62537956237793]]


In [23]:
# create list of all 2022 stock closing price
def price2022(list):
    new_list = []
    for ticker in list:
        data = [ticker, float(df_portfolio_2022["Adj Close"][ticker])]  
        new_list.append(data)
    print(new_list)

tickers_2022 = price2022(tickers)
tickers_2022

[['TMUS', 135.52000427246094], ['IDXX', 334.0299987792969], ['AMD', 68.36000061035156], ['ADP', 231.9499969482422], ['BIIB', 276.6099853515625], ['META', 141.61000061035156], ['PEP', 168.6699981689453], ['ALGN', 212.10000610351562], ['PANW', 166.8000030517578], ['AAPL', 149.83999633789062], ['ROST', 87.01000213623047], ['DXCM', 83.33000183105469], ['AMZN', 118.01000213623047], ['VRSK', 173.72999572753906], ['AMGN', 230.97999572753906], ['ORLY', 717.6300048828125], ['VRTX', 292.4100036621094], ['TSLA', 287.80999755859375], ['LULU', 309.3399963378906], ['BIDU', 121.4000015258789], ['XEL', 68.91000366210938], ['NXPI', 156.44000244140625], ['ASML', 441.2300109863281], ['ILMN', 201.2899932861328], ['SIRI', 5.949999809265137], ['ADSK', 190.97999572753906], ['FTNT', 49.970001220703125], ['KLAC', 315.69000244140625], ['GILD', 63.93000030517578], ['DLTR', 137.97000122070312], ['MDLZ', 57.75499725341797], ['ODFL', 256.17999267578125], ['SPLK', 82.05000305175781], ['NVDA', 127.36000061035156], ['

In [24]:
df = pd.DataFrame(tickers_2022, columns=['ticker', 'price'])
display(df)


,ticker,price
